In [1]:
import os
import numpy as np
import pandas as pd
from testMail import Mail

import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.linear_model import Lasso

In [2]:
mail = Mail("rhkd865@gmail.com", "drbhcuuccubulmkd")

# Data Load

In [5]:
base = os.getcwd()
data_path = os.path.join(base, 'data')
save_data_path = os.path.join(data_path, 'sub')
submit_path = os.path.join(base, 'submit')
model_path = os.path.join(base, 'model')

if not os.path.isdir(save_data_path):
    os.mkdir(save_data_path)

def load_data(name):
    return np.load(os.path.join(data_path, f"{name}.npy"))

def load_y_data(name):
    return np.load(os.path.join(save_data_path, f"{name}.npy"))

def save_data(data, name):
    np.save(os.path.join(save_data_path, f"{name}.npy"), data)

def reshape(data):
    return data.reshape(data.shape[0] * 40 * 40, data.shape[-1])

def get_test_data():
    return reshape(load_data('test'))

In [13]:
data = reshape(load_data('dl_train'))
print("data load")

data load


In [14]:
X = data[:, :-1]
Y = data[:,  -1].reshape(X.shape[0], 1)
data = range(X.shape[0])
print(X.shape, Y.shape)

(121561600, 14) (121561600, 1)


# lightgbm

In [6]:
lgb_clf = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=- 1, learning_rate=0.01, \
                        n_estimators=800, subsample_for_bin=200000, objective=None, class_weight=None, \
                        min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, \
                        subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, \
                        random_state=7, n_jobs=- 1, silent=True, importance_type='split')

In [7]:
lgb_clf.fit(X, Y, verbose=True)
print("lightbgm fit")

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


lightbgm fit


In [8]:
save_data(lgb_clf.predict(X), "lgb_train")
save_data(lgb_clf.predict(get_test_data()), "lgb_test")
print("lightbgm save data")

lightbgm save data


In [9]:
mail.send("rhkd865@gmail.com", "AIFrenz2 LightGBM", "Finished LightGBM")
mail.send("rhkd865@naver.com", "AIFrenz2 LightGBM", "Finished LightGBM")

SMTPSenderRefused: (451, b'4.4.2 Timeout - closing connection. m25sm2313168qkg.83 - gsmtp', 'rhkd865@gmail.com')

# catboost

In [10]:
cat_clf = CatBoostRegressor(iterations=500, learning_rate=0.01, \
                        depth=4, l2_leaf_reg=20, \
                        bootstrap_type='Bernoulli', subsample=0.6, \
                        eval_metric='MAE', metric_period=10, \
                        od_type='Iter', od_wait=45, random_seed=17,\
                        allow_writing_files=False)

In [11]:
cat_clf.fit(X, Y, \
        cat_features=[], verbose=True)
print("catboost fit")

0:	learn: 0.2714483	total: 2.52s	remaining: 20m 55s
10:	learn: 0.2567811	total: 23.6s	remaining: 17m 30s
20:	learn: 0.2443271	total: 44.4s	remaining: 16m 51s
30:	learn: 0.2342386	total: 1m 5s	remaining: 16m 28s
40:	learn: 0.2254261	total: 1m 26s	remaining: 16m 4s
50:	learn: 0.2178833	total: 1m 46s	remaining: 15m 40s
60:	learn: 0.2111871	total: 2m 7s	remaining: 15m 15s
70:	learn: 0.2053922	total: 2m 27s	remaining: 14m 53s
80:	learn: 0.2001924	total: 2m 48s	remaining: 14m 31s
90:	learn: 0.1957180	total: 3m 8s	remaining: 14m 9s
100:	learn: 0.1916865	total: 3m 29s	remaining: 13m 48s
110:	learn: 0.1881214	total: 3m 50s	remaining: 13m 26s
120:	learn: 0.1847778	total: 4m 10s	remaining: 13m 5s
130:	learn: 0.1820089	total: 4m 31s	remaining: 12m 43s
140:	learn: 0.1794553	total: 4m 51s	remaining: 12m 21s
150:	learn: 0.1768213	total: 5m 11s	remaining: 12m
160:	learn: 0.1743954	total: 5m 32s	remaining: 11m 39s
170:	learn: 0.1721363	total: 5m 51s	remaining: 11m 17s
180:	learn: 0.1696614	total: 6m 12

In [12]:
save_data(cat_clf.predict(X), "cat_train")
save_data(cat_clf.predict(get_test_data()), "cat_test")
print("catboost save")

catboost save


In [ ]:
mail.send("rhkd865@gmail.com", "AIFrenz2 Catboost", "Finished Catboost")
mail.send("rhkd865@naver.com", "AIFrenz2 Catboost", "Finished Catboost")

# xgboost

In [13]:
xgb_clf = xgb.XGBRegressor(max_depth=8, n_estimators=500, learning_rate=0.01,verbosity=1, silent=None, \
                           objective='reg:linear', booster='gbtree', n_jobs=1, nthread=None, gamma=0, \
                           min_child_weight=1, max_delta_step=0, subsample=1, \
                           colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, \
                           scale_pos_weight=1, base_score=0.5, random_state=0, seed=7, missing=None, importance_type='gain')

In [ ]:
xgb_clf.fit(X, Y, \
            sample_weight=None, xgb_model=None, \
            sample_weight_eval_set=None, \
            verbose=True, callbacks=None)
print("xgboost fit")

/home/ubuntu/anaconda3/lib/python3.7/site-packages/xgboost/core.py:383: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


[12:30:31] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[12:30:51] WARNING: /workspace/src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [ ]:
save_data(xgb_clf.predict(X), "xgb_train")
save_data(xgb_clf.predict(get_test_data()), "xgb_test")
print("xgboost save")

In [ ]:
mail.send("rhkd865@gmail.com", "AIFrenz2 XGBoost", "Finished XGBoost")
mail.send("rhkd865@naver.com", "AIFrenz2 XGBoost", "Finished XGBoost")

# Stack

In [10]:
names = ["lgb", "cat"]
types = ["_train", "_test"]

def data_join(i):
    return np.hstack((load_y_data(names[0] + types[i]), \
                      load_y_data(names[1] + types[i]))).reshape(-1, len(names))

In [15]:
stack_clf = Lasso()
print("stack load ")

stack load 


In [16]:
stack_clf.fit(data_join(0), Y)
print("stack fit")

stack fit


In [ ]:
mail.send("rhkd865@gmail.com", "AIFrenz2 Stacking", "Finished Stacking")
mail.send("rhkd865@naver.com", "AIFrenz2 Stacking", "Finished Stacking")

# Submit

In [17]:
def submit(clf, name):
    pred = clf.predict(data_join(1))

    submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
    submission.iloc[:, 1:] = pred.reshape(-1, 1600)

    submission.to_csv(os.path.join(submit_path, f'{name}.csv'), index=False)

In [18]:
print("Submit")
submit(stack_clf, "stack_lgb_cat")

Submit


In [ ]:
mail = Mail("rhkd865@gmail.com", "drbhcuuccubulmkd")
mail.send("rhkd865@gmail.com", "AIFrenz2", "Finished Submit")
mail.send("rhkd865@naver.com", "AIFrenz2", "Finished Submit")
mail.close()

# X Stack

In [ ]:
data = reshape(load_data('dl_train'))
X = data[:, :-1]
Y = data[:,  -1].reshape(X.shape[0], 1)
data = range(X.shape[0])
print(X.shape, Y.shape)

In [19]:
def data_join(i):
    return np.hstack((X, \
                      load_y_data(names[0] + types[i]), \
                      load_y_data(names[1] + types[i]))).reshape(-1, len(names) + X.shape[-1])

def submit(clf, name):
    pred = clf.predict(data_join(1))

    submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
    submission.iloc[:, 1:] = pred.reshape(-1, 1600)

    submission.to_csv(os.path.join(submit_path, f'{name}.csv'), index=False)

In [22]:
X.shape

(121561600, 14)

In [21]:
np.hstack((X, \
           load_y_data(names[0] + types[0]))).shape

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [20]:
stack_clf = Lasso()
stack_clf.fit(data_join(0), Y)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
submit(stack_clf, "stack_x_lgb_cat")

In [ ]:
mail = Mail("rhkd865@gmail.com", "drbhcuuccubulmkd")
mail.send("rhkd865@gmail.com", "AIFrenz2", "Finished X_Stack Submit")
mail.send("rhkd865@naver.com", "AIFrenz2", "Finished X_Stack Submit")
mail.close()